In [52]:
# Initial Setup
import pandas as pd
import numpy as np

# datavis
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno

from pathlib import Path
from scipy import stats
import os
import io

In [53]:
# set up graphing defaults to look pretty
sns.set_palette('pastel')
sns.set_theme(rc={'axes.facecolor':'lightgrey'})

# Formula One Data Analysis
Ultimate goal: build a model that will predict which constructor will win the race, given the data we have

## Data Cleaning/Wrangling

In [54]:
# Data taken from this kaggle dataset: https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020
dfs = {dir[:-4]:pd.read_csv(Path('Data') / dir) for dir in os.listdir('Data')}
circuits = dfs['circuits']
constructor_results = dfs['constructor_results']
constructor_standings = dfs['constructor_standings']
driver_standings = dfs['driver_standings']
drivers = dfs['drivers']
lap_times = dfs['lap_times']
pit_stops = dfs['pit_stops']
qualifying = dfs['qualifying']
races = dfs['races']
results = dfs['results']
seasons = dfs['seasons']
sprint_results = dfs['sprint_results']
status = dfs['status']

### Circuits
`'circuits'` contains info about the tracks (circuits) on which the races were held. It includes latitude/longitude, altitude, and country for each circuit that was raced on from 1950 - 2024.

In [55]:
circuits.head(3)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.7380,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.5106,7,http://en.wikipedia.org/wiki/Bahrain_Internati...


In [56]:
circuits['alt'].unique()

array(['10', '18', '7', '109', '130', '13', '228', '153', '103', '264',
       '4', '401', '162', '583', '5', '785', '223', '578', '37', '45',
       '\\N', '3', '8', '266', '58', '1460', '88', '2227', '345', '432',
       '0', '1126', '177', '145', '6', '36', '484', '139', '12', '639',
       '609', '485', '332', '79', '790', '214', '81', '67', '15', '676',
       '20', '28', '470', '53', '158', '19', '129', '551', '85', '194',
       '161', '678', '2', '-7', '108', '255'], dtype=object)

This is pretty clean already. It just looks like altitude has some `NaN`'s represented as '\\N'. Let's replace those with actual `NaN`.

In [57]:
circuits['alt'] = circuits['alt'].replace('\\N', np.NaN)
circuits['alt'].unique()

array(['10', '18', '7', '109', '130', '13', '228', '153', '103', '264',
       '4', '401', '162', '583', '5', '785', '223', '578', '37', '45',
       nan, '3', '8', '266', '58', '1460', '88', '2227', '345', '432',
       '0', '1126', '177', '145', '6', '36', '484', '139', '12', '639',
       '609', '485', '332', '79', '790', '214', '81', '67', '15', '676',
       '20', '28', '470', '53', '158', '19', '129', '551', '85', '194',
       '161', '678', '2', '-7', '108', '255'], dtype=object)

We probably won't be using the urls from this dataset, so let's just drop those and finish up our cleaning.

In [58]:
circuits = circuits.drop('url', axis=1)

In [59]:
circuits.head(3)

,circuitId,circuitRef,name,location,country,lat,lng,alt
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.7380,18
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.5106,7


### Constructor Results

In [60]:
constructor_results.head()

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,\N
1,2,18,2,8.0,\N
2,3,18,3,9.0,\N
3,4,18,4,5.0,\N
4,5,18,5,2.0,\N


In [63]:
constructor_results['status'].unique()

array(['\\N', 'D'], dtype=object)

The 'status' column seems to be entirely misssing, so let's just save the headache and drop it.

In [64]:
constructor_results[constructor_results['status'] == 'D']

,constructorResultsId,raceId,constructorId,points,status
185,186,36,1,14.0,D
195,196,37,1,18.0,D
207,208,38,1,12.0,D
218,219,39,1,14.0,D
228,229,40,1,18.0,D
239,240,41,1,12.0,D
250,251,42,1,18.0,D
262,263,43,1,8.0,D
273,274,44,1,14.0,D
283,284,45,1,10.0,D


### Races
The `'races'` dataframe contains info about each race, including the circuit, date, and time. The `'fp'` columns contain the dates and times of the Free Practices.

In [76]:
races.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


### Results
The `'results'` dataframe has the meat of our data: The results of each race, for each driver. This includes car number for each driver (`'number'`), position in the starting grid (`'grid'`), final position (`'position'`), final rank (`'positionOrder'`), and data on lap speeds. Speeds are (probably) in km/h.

For `'position text'`, the values are as follows:
- Any integer: finishing position
- `R`: retired
- `D`: disqualified
- `E`: excluded
- `W`: withdrawn
- `F`: failed to qualify
- `N`: not classified

In [77]:
results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


Taking a look at this dataset, the null values are also represented with '\N', so we'll replace those with proper NaNs.

In [79]:
results = results.replace('\\N', np.NaN)

Since we have the data for their final times in milliseconds, we don't really need the `'time'` column - it's just giving us the same data, but is harder to work with due to its inconsistent formatting. Let's drop it.

In [81]:
results = results.drop('time', axis=1)
results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,5708630,43,1,1:27.418,218.385,1


In [84]:
results['positionText'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', 'R', 'D', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', 'N',
       'W', 'F', 'E', '22', '23', '24', '25', '26', '27', '28', '29',
       '30', '31', '32', '33'], dtype=object)